#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [ ]:
!pip install -q aitextgen

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Mon May 17 03:18:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

# Comment out the above line and uncomment the below line to use GPT Neo instead.
# ai = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

07/28/2021 10:10:29 — INFO — aitextgen — Downloading the 124M GPT-2 TensorFlow weights/config from Google's servers


07/28/2021 10:11:01 — INFO — aitextgen — Converting the 124M GPT-2 TensorFlow weights to PyTorch.


Converting TensorFlow checkpoint from /content/aitextgen/124M
Loading TF weight model/h0/attn/c_attn/b with shape [2304]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_proj/b with shape [768]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 768, 768]
Loading TF weight model/h0/ln_1/b with shape [768]
Loading TF weight model/h0/ln_1/g with shape [768]
Loading TF weight model/h0/ln_2/b with shape [768]
Loading TF weight model/h0/ln_2/g with shape [768]
Loading TF weight model/h0/mlp/c_fc/b with shape [3072]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 768, 3072]
Loading TF weight model/h0/mlp/c_proj/b with shape [768]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 3072, 768]
Loading TF weight model/h1/attn/c_attn/b with shape [2304]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h1/attn/c_proj/b with shape [768]
Loading TF weight model/h1/attn/c_proj/w with shape [1, 7

Save PyTorch model to aitextgen/pytorch_model.bin


07/28/2021 10:11:07 — INFO — aitextgen — Loading 124M GPT-2 model from /aitextgen.


Save configuration file to aitextgen/config.json


07/28/2021 10:11:08 — INFO — aitextgen — GPT2 loaded with 124M parameters.
07/28/2021 10:11:08 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "file_name.txt"

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [ ]:
copy_file_from_gdrive(file_name)

In [ ]:
file1 = open ("/content/drive/MyDrive/messages/messages20.html", "r")
file2 = open ("/content/drive/MyDrive/messages/messages19.html", "r")
file3 = open ("/content/drive/MyDrive/messages/messages18.html", "r")
file4 = open ("/content/drive/MyDrive/messages/messages17.html", "r")
file5 = open ("/content/drive/MyDrive/messages/messages16.html", "r")
file6 = open ("/content/drive/MyDrive/messages/messages15.html", "r")
file7 = open ("/content/drive/MyDrive/messages/messages14.html", "r")
file8 = open ("/content/drive/MyDrive/messages/messages13.html", "r")
file9 = open ("/content/drive/MyDrive/messages/messages12.html", "r")
file10 = open ("/content/drive/MyDrive/messages/messages11.html", "r")
file11= open ("/content/drive/MyDrive/messages/messages10.html", "r")
file12 = open ("/content/drive/MyDrive/messages/messages9.html", "r")
file13 = open ("/content/drive/MyDrive/messages/messages8.html", "r")
file14 = open ("/content/drive/MyDrive/messages/messages7.html", "r")
file15 = open ("/content/drive/MyDrive/messages/messages6.html", "r")
file16 = open ("/content/drive/MyDrive/messages/messages5.html", "r")
file17 = open ("/content/drive/MyDrive/messages/messages4.html", "r")
file19 = open ("/content/drive/MyDrive/messages/messages3.html", "r")
file18= open ("/content/drive/MyDrive/messages/messages2.html", "r")
file21 = open ("/content/drive/MyDrive/messages/messages.html", "r")
data = []
while True:
    # считываем строку
    line = file1.readline()
    line1 = file2.readline()
    line2 = file3.readline()
    line3 = file4.readline()
    line4 = file5.readline()
    line5 = file6.readline()
    line6 = file7.readline()
    line7 = file8.readline()
    line8 = file9.readline()
    line9 = file10.readline()
    line10 = file11.readline()
    line11 = file12.readline()
    line12 = file13.readline()
    line13 = file14.readline()
    line14 = file15.readline()
    line15 = file16.readline()
    line16 = file17.readline()
    line17 = file18.readline()
    line18 = file19.readline()
    line20 = file21.readline()

    # прерываем цикл, если строка пустая
    if not line:
        break
    # выводим строку
    data.append (line.strip())
    data.append (line1.strip())
    data.append (line2.strip())
    data.append (line3.strip())
    data.append (line4.strip())
    data.append (line5.strip())
    data.append (line6.strip())
    data.append (line7.strip())
    data.append (line8.strip())
    data.append (line9.strip())
    data.append (line10.strip())
    data.append (line11.strip())
    data.append (line12.strip())
    data.append (line13.strip())
    data.append (line14.strip())
    data.append (line15.strip())
    data.append (line16.strip())
    data.append (line17.strip())
    data.append (line18.strip())
    data.append (line20.strip())


import re 

r = re.compile("[а-яА-Я]+", )
russian = [w for w in filter(r.match, data)]

names = ['ач' 'НО', "лб", "Е", "тк", "ИМ", "С", "К", "ак", "В", "МЩ", "ИП"]
final = [item for item in russian if item not in names]
final1 = [sent for sent in final 
    if not any(word in sent for word in 'http')]
with open("file_name.txt", "w") as output:
    output.write(str(final1))


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [ ]:
ai.train(file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=3000,
         generate_every=1000,
         save_every=1000,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

07/28/2021 10:25:54 — INFO — aitextgen — Loading text from file_name.txt with generation length of 1024.


07/28/2021 10:25:54 — INFO — aitextgen.TokenDataset — Encoding 1 sets of tokens from file_name.txt.
07/28/2021 10:25:54 — INFO — pytorch_lightning.utilities.distributed — GPU available: True, used: True
07/28/2021 10:25:54 — INFO — pytorch_lightning.utilities.distributed — TPU available: False, using: 0 TPU cores
07/28/2021 10:25:54 — INFO — pytorch_lightning.utilities.distributed — IPU available: False, using: 0 IPUs
07/28/2021 10:25:54 — INFO — pytorch_lightning.accelerators.gpu — LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
', 'Коля солько фитраказиять вообщем ком', 'Аха', 'Я сперошила вечем', 'я себезяругая как на отпизите', 'Ну не толькое всетых-то жно в начик, все выпридится)', 'Мнезвоно не нат проднято сасоя', 'Сопро в от стобрько забака', 'Да ста', 'Так ебата))', '
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
', 'Зам умралену выщи заможной получает)', 'НО', 'Заченлил', 'А че к силавись буца посылкаты на соямбраная работлемонимать', 'Давай в вообще домаресно', 'Ну тебереги красты)', 'Максимся возронаять на отдыхают дорежалось пла у нас ебанять из пока
3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
 'Чтоб чего', 'Ну чтоб когда потивать на восегде понилось надеюсь', 'У меня када подками', 'Вован я сперкер живединуть пока из минут, но хуй контой ещё не подстать ( ага', 'Ахах', 'Мы вообще еще дачем черего',

07/28/2021 11:02:12 — INFO — aitextgen — Saving trained model pytorch_model.bin to /trained_model


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


## Load a Trained Model

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
from_folder = None

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

In [ ]:
ai = aitextgen(model_folder=".", to_gpu=True)

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

In [ ]:
ai = aitextgen(model_folder="trained_model", to_gpu=True)

04/19/2021 01:36:04 — INFO — aitextgen — Loading model from provided weights and config in /trained_model.
04/19/2021 01:36:09 — INFO — aitextgen — GPT2 loaded with 124M parameters.
04/19/2021 01:36:09 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


`generate()` without any parameters generates a single text from the loaded model to the console.

In [ ]:
ai.generate()

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [ ]:
ai.generate(n=5,
            batch_size=5,
            prompt="Добрые мутки",
            max_length=256,
            temperature=0.7,
            top_p=0.9)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_files = 5

for _ in range(num_files):
  ai.generate_to_file(n=1000,
                     batch_size=50,
                     prompt="",
                     max_length=256,
                     temperature=0.7,
                     top_p=0.9)